In [6]:
import os
import glob
import numpy as np
import skvideo.io
import torchio as tio
from onnxruntime.quantization import quantize_static, CalibrationDataReader, QuantType

class SegmentationCalibrationReader(CalibrationDataReader):
    def __init__(self, video_dir, num_samples=5):
        self.video_paths = sorted(glob.glob(os.path.join(video_dir, "*.mp4")))[:num_samples]
        self.enum_data_dicts = []
        self.preprocess_flag = True

    def preprocess_video(self, path):
        video = skvideo.io.vread(path)
        D, H, W, _ = video.shape
        cx, cy = W // 2, H // 2
        video = video[:, cy - 456:cy + 456, cx - 456:cx + 456, :]

        gray = np.zeros((1, D, video.shape[1], video.shape[2]), dtype=np.uint8)
        for i in range(D):
            gray[0, i] = np.dot(video[i], [0.2989, 0.5870, 0.1140])

        img = tio.ScalarImage(tensor=gray)
        resized = tio.Resize((128, 128, 128))(img).data.numpy()
        return np.expand_dims(resized, axis=0).astype(np.float32)  # (1, 1, D, H, W)

    def get_next(self):
        if self.preprocess_flag:
            self.enum_data_dicts = []
            for path in self.video_paths:
                input_data = self.preprocess_video(path)
                self.enum_data_dicts.append({"input": input_data})
            self.enum_data_dicts = iter(self.enum_data_dicts)
            self.preprocess_flag = False
        return next(self.enum_data_dicts, None)

# Cuantización por lotes
video_dir = "./videos"
onnx_dir = "./onnx_models_preprocessed"
out_dir = "./onnx_models_quantized_preprocessed"
num_calib = 6

os.makedirs(out_dir, exist_ok=True)
model_paths = sorted(glob.glob(os.path.join(onnx_dir, "model_fold*.onnx")))

for model_path in model_paths:
    model_name = os.path.basename(model_path)
    print(f"🔍 Procesando: {model_name}")
    out_path = os.path.join(out_dir, model_name.replace(".onnx", "_int8.onnx"))

    print(f"🔄 Cuantizando: {model_name} → {os.path.basename(out_path)}")

    reader = SegmentationCalibrationReader(video_dir, num_samples=num_calib)

    quantize_static('/Users/emilio/Library/CloudStorage/Box-Box/GitHub/Breast-AI-model/src/onnx_models_preprocessed/'+model_name,
                out_path,
                reader)

    print(f"✅ Guardado: {out_path}")


🔍 Procesando: model_fold0_preproc.onnx
🔄 Cuantizando: model_fold0_preproc.onnx → model_fold0_preproc_int8.onnx
✅ Guardado: ./onnx_models_quantized_preprocessed/model_fold0_preproc_int8.onnx
🔍 Procesando: model_fold1_preproc.onnx
🔄 Cuantizando: model_fold1_preproc.onnx → model_fold1_preproc_int8.onnx
✅ Guardado: ./onnx_models_quantized_preprocessed/model_fold1_preproc_int8.onnx
🔍 Procesando: model_fold2_preproc.onnx
🔄 Cuantizando: model_fold2_preproc.onnx → model_fold2_preproc_int8.onnx
✅ Guardado: ./onnx_models_quantized_preprocessed/model_fold2_preproc_int8.onnx
🔍 Procesando: model_fold3_preproc.onnx
🔄 Cuantizando: model_fold3_preproc.onnx → model_fold3_preproc_int8.onnx
✅ Guardado: ./onnx_models_quantized_preprocessed/model_fold3_preproc_int8.onnx
🔍 Procesando: model_fold4_preproc.onnx
🔄 Cuantizando: model_fold4_preproc.onnx → model_fold4_preproc_int8.onnx
✅ Guardado: ./onnx_models_quantized_preprocessed/model_fold4_preproc_int8.onnx
🔍 Procesando: model_fold5_preproc.onnx
🔄 Cuantizan